## Normalization

In [ ]:
from connectome import Transform


class Normalize(Transform):
    __inherit__ = True

    def image(image, padding_value, intensity_sign):
        if padding_value is not None:
            if padding_value > 0:
                return padding_value - image
            return image

        if intensity_sign == 1:
            return image.max() - image

        return image

## Zoom to reduce image size

In [ ]:
from connectome import Apply
from scipy.ndimage import zoom

# 0.25 - is the downsample factor. It should probably be tuned via cross-validation
Zoom = Apply(image=lambda x: zoom(np.float32(x), 0.25, order=1))

## Artifacts and background removal

In [ ]:
from connectome import Transform
from skimage.morphology import label


class GreatestComponent(Transform):
    __inherit__ = True

    def image(image):
        lbl = label(image > 0)
        values, counts = np.unique(lbl, return_counts=True)
        foreground = values != 0
        component = values[foreground][counts[foreground].argmax()]
        # select all the components greater than the background
        #  + the greatest foreground component
        components = set(values[counts > counts[~foreground]]) | {component}
        if len(components) > 1:
            # if there are several components - pick the one with the greatest intensity
            component = max(components, key=lambda c: image[lbl == c].mean())

        return image * (lbl == component)


class CropBackground(Transform):
    __inherit__ = True

    def image(image):
        mask = image > 0
        xs, = mask.any(0).nonzero()
        ys, = mask.any(1).nonzero()
        return image[ys.min():ys.max() + 1, xs.min():xs.max() + 1]

## Data augmentation

In [ ]:
from connectome import Transform, impure
import numpy as np


class RandomFlip(Transform):
    __inherit__ = True

    @impure
    def _flip():
        return np.random.binomial(1, 0.5)

    def image(image, _flip):
        if _flip:
            return np.flip(image, axis=1)
        return image

## Combining it all together

In [ ]:
from amid.rsna_bc import RSNABreastCancer
from connectome import Chain

ds = Chain(
    RSNABreastCancer('/path/to/downloaded/folder'),
    Normalize(),
    Apply(image=lambda x: zoom(np.float32(x), 0.25, order=1)),
    GreatestComponent(),
    CropBackground(),

    # aug
    RandomFlip(),
)